# Il giro del mondo in 80 giorni

Partendo dai dati contenuti nel file `./data/worldcities_ascii.csv` trovare per ogni città le 3 città più vicine ad essa.

Calcolare il tempo che ci vuole a spostarsi da una città a un suo vicino secondo queste regole:

- 2 ore per la città più vicina
- 4 ore per la seconda città più vicina
- 8 ore per la terza città più vicina

Inoltre il viaggio richiede:

- 2 ore aggiuntive se la città di destinazione è in un'altra nazione rispetto alla città di partenza
- 2 ore aggiuntive se la città di destinazione ha più di 200.000 abitanti

## Quesiti

1. Partendo da Londra e viaggiando sempre verso Est, è possible compiere il giro del mondo tornando a Londra in 80 giorni? Quanto tempo si impiega al minimo?

2. In caso non si possa compiere il giro del mondo partendo da Londra, da quali altre città è possibile compiere il giro del mondo?

3. Supponendo che da ogni città si possa viaggiare solo verso le 3 città a Est più vicine come cambia l'esito delle domande precedenti?

In [1]:
import numpy as np
import pandas as pd
from dijkstra import DijkstraSPF, Graph
from sklearn.neighbors import BallTree
import math
import csv
from ipyleaflet import Map, basemaps, Marker, Polyline, TileLayer, WidgetControl, FullScreenControl, AwesomeIcon, AntPath, LegendControl
from ipywidgets import Combobox, Output

## Preprocessing

In [2]:
data_file = './data/worldcities_ascii.csv'
with open(data_file, 'r') as f:
    reader = csv.reader(f)
    all_lines = []
    for row in reader:
        all_lines.append(row)

In [3]:
newLines = []
for line in all_lines:
    newLine = []
    for index, field in enumerate(line):
        if ("." in field) and (index == 5): #Toglie il '.' nei valori della popolazione
            field = field.replace(".", "")
        field = field.replace("Korea, South", "South Korea")
        field = field.replace("Korea, North", "North Korea")
        field = field.replace("Gambia, The", "The Gambia")
        field = field.replace("Micronesia, Federated States Of", "Federated States Of Micronesia")
        field = field.replace("Bahamas, The", "The Bahamas")
        field = field.replace("Saint Helena, Ascension, And Tristan Da Cunha", "Saint Helena, Ascension and Tristan da Cunha")
        field = field.replace("Islamorada, Village of Islands", "Village of Islands Islamorada")
        if index == 6:
            field = field.replace("\n", "")
        newLine.append(field)
    newLines.append(newLine)

df = pd.DataFrame(newLines[1:], columns=newLines[0])
df = df.astype({"lat": float, "lng": float, "id": int})
df

,city,lat,lng,country,iso3,population,id
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140
...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159


## Per ogni città trovare le 3 città più vicine

Come struttura di supporto utilizzo un BallTree, struttura dati ottimale per cercare il punto più vicino a un altro dato un insieme di punti

In [4]:
coords = [np.array([c.lat, c.lng])*math.pi/180 for _, c in df.iterrows()]
tree = BallTree(coords, leaf_size=4, metric="haversine")

Creo la funzione che calcola il tempo necessario a spostarsi tra due città

In [5]:
def distanceTime(start, end, neighbour_pos: int):
    if neighbour_pos == 0:
        time = 2
    elif neighbour_pos == 1:
        time = 4
    elif neighbour_pos == 2:
        time = 8
    if start.iso3 != end.iso3:
        time += 2
    if (end.population != '') and (int(end.population) > 200000):
        time += 2
    return time

Infine rappresento le città con un grafo direzionale pesato dove il peso di ogni arco corrisponde alle ore che ci vogliono a spostarsi tra due nodi.

In [6]:
graph = Graph()
neighbours = list()
for i in df.index:
    n = tree.query(coords[i:i+1], return_distance=False, breadth_first=True, k=4)[0,1:]
    
    graph.add_edge(i, n[0], distanceTime(df.iloc[i], df.iloc[n[0]], 0))
    graph.add_edge(i, n[1], distanceTime(df.iloc[i], df.iloc[n[1]], 1))
    graph.add_edge(i, n[2], distanceTime(df.iloc[i], df.iloc[n[2]], 2))
    
    neighbours.append(n)
df_neigh = df.assign(neighbours=neighbours)
df_neigh

,city,lat,lng,country,iso3,population,id,neighbours
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764,"[4105, 2663, 1196]"
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077,"[336, 451, 346]"
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604,"[785, 337, 523]"
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629,"[13002, 563, 946]"
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140,"[1497, 3456, 461]"
...,...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709,"[26029, 26504, 26012]"
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491,"[26497, 8478, 25993]"
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438,"[14152, 18135, 21854]"
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159,"[26453, 26336, 25455]"


In [7]:
class MapWrapper:
    
    def __init__(self, mapInstance: Map, dataset: pd.DataFrame, cityList: Combobox):
        self.map = mapInstance
        self.map.layout.height = "600px"
        self.df = dataset
        self.cityList = cityList
        
        self.map.add_control(WidgetControl(widget=self.cityList, position='topright'))
        self.map.add_control(FullScreenControl())
        
    def drawMarker(self, city, icon=AwesomeIcon()):
        marker = Marker(location=(city.lat, city.lng), draggable=False, title=city.city, icon=icon)
        #marker.on_click(lambda **a: add_marker(df.iloc[193]))
        self.map.add_layer(marker)
    
    def drawLine(self, coords, color="green"):
        line = Polyline(
            locations=coords,
            color=color,
            fill=False
        )
        self.map.add_layer(line)

    def draw_neighbours(self, city):
        self.drawMarker(city)
        for neigh_index in city.neighbours:
            neigh = self.df.iloc[neigh_index]
            self.drawMarker(neigh, AwesomeIcon(name='circle', marker_color='green'))
            coords = [
                [city.lat, city.lng],
                [neigh.lat, neigh.lng]
            ]
            self.drawLine(coords)
            
    def drawPath(self, cityIndex):
        self.map.zoom = 2
        self.map.center = (0,0)
        
        city = self.df.iloc[cityIndex]
        minPath = []
        minDist = float('inf')
        for neighIndex in city.neighbours:
            dijkstra = DijkstraSPF(self.graph, neighIndex)
            dist = dijkstra.get_distance(cityIndex) + DijkstraSPF(self.graph, neighIndex).get_distance(neighIndex)
            if dist < minDist:
                minDist = dist
                minPath = dijkstra.get_path(cityIndex)

        if len(minPath) == 0:
            self.drawMarker(city, AwesomeIcon(name="times", marker_color="red"))
            self.map.zoom = 8
            self.map.center = (city.lat, city.lng)
            return

        coordsPath = []
        for index in minPath:
            city = self.df.iloc[index]
            if index == cityIndex:
                self.drawMarker(city)
            else:
                self.drawMarker(city, AwesomeIcon(name='circle', marker_color='green'))
            coordsPath.append([city.lat, city.lng])
        self.drawAntPath(coordsPath)
        
        self.map.zoom = 8
        self.map.center = (self.df.iloc[cityIndex].lat, self.df.iloc[cityIndex].lng)
        
        minPath.insert(0, cityIndex)
        return self.getPathDataFrame(minPath)   
    
    def drawAntPath(self, coords):
        coords.sort(key=lambda c: c[1])
        self.map.add_layer(AntPath(
            locations=coords,
            dash_array=[1, 10],
            delay=1000,
            color='#ff6600',
            pulse_color='#ffffff'
        ))
        
    def getPathDataFrame(self, path):
        pathDataFrame = pd.DataFrame([[self.df.city[path[0]], "0 giorni e 0 ore"]], columns=["Città", "Durata"])
        totalTripTime = 0
        for index, cityIndex in enumerate(path[1:]):
            cityName = self.df.city[cityIndex]
            totalTripTime += self.graph.get_edge_weight(path[index], cityIndex)
            timeString = f"{totalTripTime//24} giorni e {totalTripTime%24} ore"
            appendMe = pd.DataFrame([[cityName, timeString]], columns=["Città", "Durata"])
            pathDataFrame = pd.concat([pathDataFrame, appendMe], ignore_index=True)
        return pathDataFrame

    def clearMap(self):
        self.map.layers = (self.map.layers[0],)
    
    def display(self):
        display(self.map)
        
class MapNeighbours(MapWrapper):
    
    def __init__(self, mapInstance: Map, dataset: pd.DataFrame, cityList: Combobox):
        super().__init__(mapInstance, dataset, cityList)

        def callback(args):
            if args["new"] == "":
                return
            self.clearMap()
            index = int(args["new"].split("- ")[-1])
            city = self.df.iloc[index]
            self.map.center = (city.lat, city.lng)
            self.map.zoom = 8
            self.draw_neighbours(city)
            self.cityList.value = ""

        self.cityList.observe(callback, names="value")

In [8]:
options = [f"{city.city} ({city.iso3}) - {i}" for i, city in df.iterrows()]

In [9]:
m = Map(basemap=basemaps.OpenStreetMap.HOT, zoom=2, scroll_wheel_zoom=True, world_copy_jump=True, attribution_control=False)
cityList = Combobox(options=options, placeholder='ex. London', description='City:', ensure_option=True)
map1 = MapNeighbours(m, df_neigh, cityList)
map1.display()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Quesito #1

> Partendo da Londra e viaggiando sempre verso Est, è possible compiere il giro del mondo tornando a Londra in 80 giorni? Quanto tempo si impiega al minimo?

L'algoritmo utilizzato per trovare il percorso minimo tra due nodi in un grafo è Dijkstra. Per imporre di cercare solo a Est ricreo il grafo creando un arco solo se il vicino di una città si trova più ad Est.

In [10]:
eastGraph = Graph()
neighbours = list()
for i in df.index:
    n = tree.query(coords[i:i+1], return_distance=False, breadth_first=True, k=4)[0,1:]
    eastNeigh = []
    
    for neighIndex, neigh in enumerate(n):
        lngDiff = df.iloc[neigh].lng - df.iloc[i].lng
        if (0 < lngDiff < 180) or (-360 < lngDiff < -180):
            eastGraph.add_edge(i, neigh, distanceTime(df.iloc[i], df.iloc[neigh], neighIndex))
            eastNeigh.append(neigh)
    
    neighbours.append(eastNeigh)
dfEast = df.assign(neighbours=neighbours)
dfEast

,city,lat,lng,country,iso3,population,id,neighbours
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764,"[4105, 1196]"
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077,[336]
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604,"[337, 523]"
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629,"[13002, 563, 946]"
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140,"[1497, 3456]"
...,...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709,[]
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491,[]
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438,[14152]
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159,[25455]


Imponendo la condizione che tra le 3 città più vicine si possa viaggiare solo a Est il numero di vicini di molte città si è ridotto a 2, 1 o addirittura 0 (nel caso tutti e 3 i vicini si trovassero a Ovest).

In [11]:
m = Map(basemap=basemaps.OpenStreetMap.HOT, zoom=2, scroll_wheel_zoom=True, world_copy_jump=True, attribution_control=False)
cityList = Combobox(options=options, placeholder='ex. London', description='City:', ensure_option=True)
map2 = MapNeighbours(m, dfEast, cityList)
map2.display()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Trovo a che indice corrisponde Londra e controllo se Dijkstra trova un percorso che partendo da Londra mi riporti a Londra.

> Dijkstra di default dato lo stesso nodo di partenza e di arrivo restituisce 0 come distanza minima. Per ovviare a questo problema si è applicato Dijkstra partendo dai 3 vicini di Londra. In caso si trovasse un percorso bisognerebbe aggiungere anche l'hop per arrivare da Londra al suo vicino.

In [12]:
londonIndex = [index for index, city in df.iterrows() if (city.city == "London") and (city.iso3 == "GBR")][0]

In [13]:
city = map2.df.iloc[londonIndex]
for neighIndex in city.neighbours:
    dijkstra = DijkstraSPF(eastGraph, neighIndex)
    if dijkstra.get_distance(index) != float('inf'):
        print(f"Da Londra è possibile fare il giro del mondo!")

Dato che la cella precedente non ha stampato nulla possiamo affermare che **non è possibile fare il giro del mondo partendo da Londra**

## Quesito #2

> In caso non si possa compiere il giro del mondo partendo da Londra, da quali altre città è possibile compiere il giro del mondo?

Semplicemente basta applicare Dijkstra per ogni città contenuta in `df`

In [14]:
testDf = map2.df
for index, city in testDf.iterrows():
    for neighIndex in city.neighbours:
        dijkstra = DijkstraSPF(eastGraph, neighIndex)
        if dijkstra.get_distance(index) != float('inf'):
            print(f"Dalla città {city.city} è possibile fare il giro del mondo!")

Anche questa cella non ha stampato nulla quindi **non è possibile fare il giro del mondo con queste condizioni**

## Quesito #3

> Supponendo che da ogni città si possa viaggiare solo verso le 3 città a Est più vicine come cambia l'esito delle domande precedenti?

Costruisco un ulteriore grafo e per assicurarmi che ogni città abbia 3 vicini e che siano le 3 città a Est più vicine scorro i suoi vicini in ordine di distanza fermandomi quando trovo la terza città ad Est.

In [15]:
newEastGraph = Graph()
neighbours = list()
for i in df.index:
    n = tree.query(coords[i:i+1], return_distance=False, breadth_first=True, k=1000)[0,1:]
    eastNeigh = []
    
    for neigh in n:
        lngDiff = df.iloc[neigh].lng - df.iloc[i].lng
        if (0 < lngDiff < 180) or (-360 < lngDiff < -180):
            newEastGraph.add_edge(i, neigh, distanceTime(df.iloc[i], df.iloc[neigh], len(eastNeigh)))
            eastNeigh.append(neigh)
        if len(eastNeigh) == 3:
            break
    
    neighbours.append(eastNeigh)
dfNewEast = df.assign(neighbours=neighbours)
dfNewEast

,city,lat,lng,country,iso3,population,id,neighbours
0,Tokyo,35.6897,139.6922,Japan,JPN,37977000,1392685764,"[4105, 1196, 509]"
1,Jakarta,-6.2146,106.8451,Indonesia,IDN,34540000,1360771077,"[336, 1878, 1195]"
2,Delhi,28.6600,77.2300,India,IND,29617000,1356872604,"[337, 523, 2337]"
3,Mumbai,18.9667,72.8333,India,IND,23355000,1356226629,"[13002, 563, 946]"
4,Manila,14.5958,120.9772,Philippines,PHL,23088000,1608618140,"[1497, 3456, 1545]"
...,...,...,...,...,...,...,...,...
26564,Nord,81.7166,-17.8000,Greenland,GRL,10,1304217709,"[22081, 5963, 17212]"
26565,Timmiarmiut,62.5333,-42.2167,Greenland,GRL,10,1304206491,"[25969, 26474, 16692]"
26566,Cheremoshna,51.3894,30.0989,Ukraine,UKR,0,1804043438,"[14152, 20185, 2024]"
26567,Ambarchik,69.6510,162.3336,Russia,RUS,0,1643739159,"[25455, 25887, 26519]"


In [16]:
class MapWorldTrip(MapWrapper):
    
    def __init__(self, mapInstance: Map, dataset: pd.DataFrame, cityList: Combobox, graph: Graph):
        super().__init__(mapInstance, dataset, cityList)
        self.graph = graph
        legend = LegendControl({"Starting city":"blue", "Path cities":"green", "Starting city not available":"red"}, name="Legend", position="bottomright")
        self.map.add_control(legend)
        self.output = Output()
        self.map.add_control(WidgetControl(widget=self.output, position='bottomleft'))

        def callback(args):
            self.output.clear_output()
            self.clearMap()
            index = int(args["new"].split("- ")[-1])
            pathDataFrame = self.drawPath(index)
            if pathDataFrame is not None:
                with self.output:
                    pd.set_option("display.max_rows", None)
                    display(pathDataFrame)
                    pd.set_option("display.max_rows", 60)

        self.cityList.observe(callback, names="value")

In [17]:
m = Map(basemap=basemaps.OpenStreetMap.HOT, zoom=2, scroll_wheel_zoom=True, world_copy_jump=True, attribution_control=False)
cityList = Combobox(options=options, placeholder='ex. London', description='City:', ensure_option=True)
map3 = MapWorldTrip(m, dfNewEast, cityList, newEastGraph)
map3.display()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

**Da Londra non è comunque possibile fare il giro del mondo ma da molte altre città si**